# 🚀 Google Colab Setup[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ogautier1980/sandbox-ml/blob/main/cours/08_deep_learning_rnn/08_exercices_solutions.ipynb)**Si vous exécutez ce notebook sur Google Colab**, exécutez la cellule suivante pour installer les dépendances.

In [ ]:
# Installation des dépendances (Google Colab uniquement)import sysIN_COLAB = 'google.colab' in sys.modulesif IN_COLAB:    print('📦 Installation des packages...')        # Packages ML de base    !pip install -q numpy pandas matplotlib seaborn scikit-learn        # Détection du chapitre et installation des dépendances spécifiques    notebook_name = '08_exercices.ipynb'  # Sera remplacé automatiquement        # Ch 06-08 : Deep Learning    if any(x in notebook_name for x in ['06_', '07_', '08_']):        !pip install -q torch torchvision torchaudio        # Ch 08 : NLP    if '08_' in notebook_name:        !pip install -q transformers datasets tokenizers        if 'rag' in notebook_name:            !pip install -q sentence-transformers faiss-cpu rank-bm25        # Ch 09 : Reinforcement Learning    if '09_' in notebook_name:        !pip install -q gymnasium[classic-control]        # Ch 04 : Boosting    if '04_' in notebook_name and 'boosting' in notebook_name:        !pip install -q xgboost lightgbm catboost        # Ch 05 : Clustering avancé    if '05_' in notebook_name:        !pip install -q umap-learn        # Ch 11 : Séries temporelles    if '11_' in notebook_name:        !pip install -q statsmodels prophet        # Ch 12 : Vision avancée    if '12_' in notebook_name:        !pip install -q ultralytics timm segmentation-models-pytorch        # Ch 13 : Recommandation    if '13_' in notebook_name:        !pip install -q scikit-surprise implicit        # Ch 14 : MLOps    if '14_' in notebook_name:        !pip install -q mlflow fastapi pydantic        print('✅ Installation terminée !')else:    print('ℹ️  Environnement local détecté, les packages sont déjà installés.')

# Chapitre 08 - Solutions : RNN, LSTM et TransformersCe notebook contient des exercices pratiques sur les réseaux récurrents et les Transformers.## Solutions1. Seq2Seq avec Attention pour traduction2. Génération de texte avec LSTM3. Prédiction de séries temporelles avec GRU4. Classification multi-classe avec BERT

In [ ]:
import torchimport torch.nn as nnimport torch.optim as optimimport numpy as npimport matplotlib.pyplot as pltimport seaborn as snsfrom collections import Counterimport warningswarnings.filterwarnings('ignore')plt.style.use('seaborn-v0_8-darkgrid')sns.set_palette('husl')device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')print(f"Using device: {device}")

## Exercice 1: Seq2Seq avec Attention### ObjectifImplémenter un modèle Sequence-to-Sequence avec mécanisme d'attention pour traduire des nombres en mots.### Instructions1. Créer un dataset de paires (nombre, mot): (1, "one"), (2, "two"), etc.2. Implémenter un Encoder LSTM3. Implémenter un Decoder LSTM avec Attention4. Entraîner le modèle5. Tester sur de nouveaux nombres

In [ ]:
# SOLUTION: Dataset nombre -> motnumber_to_word = {    '1': 'one', '2': 'two', '3': 'three', '4': 'four', '5': 'five',    '6': 'six', '7': 'seven', '8': 'eight', '9': 'nine', '0': 'zero'}# Générer des exemples: "123" -> "one two three"def generate_samples(num_samples=1000, max_len=5):    samples = []    for _ in range(num_samples):        length = np.random.randint(1, max_len + 1)        number_str = ''.join([str(np.random.randint(0, 10)) for _ in range(length)])        word_str = ' '.join([number_to_word[d] for d in number_str])        samples.append((number_str, word_str))    return samplessamples = generate_samples(1000)print("Sample pairs:")for i in range(5):    print(f"  {samples[i][0]} -> {samples[i][1]}")

In [ ]:
# SOLUTION: Vocabulaire# Créer des vocabulaires pour input (nombres) et output (mots)# Indices spéciaux: <PAD>=0, <SOS>=1, <EOS>=2input_vocab = {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2}for digit in '0123456789':    input_vocab[digit] = len(input_vocab)output_vocab = {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2}for word in number_to_word.values():    if word not in output_vocab:        output_vocab[word] = len(output_vocab)# Vocabulaires inversésinv_output_vocab = {v: k for k, v in output_vocab.items()}print(f"Input vocab size: {len(input_vocab)}")print(f"Output vocab size: {len(output_vocab)}")print(f"Output vocab: {output_vocab}")

In [ ]:
# SOLUTION: Encoder LSTMclass Encoder(nn.Module):    def __init__(self, input_size, embedding_dim, hidden_dim):        super(Encoder, self).__init__()        self.embedding = nn.Embedding(input_size, embedding_dim)        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)        def forward(self, x):        # x: (batch_size, seq_len)        embedded = self.embedding(x)  # (batch_size, seq_len, embedding_dim)        outputs, (hidden, cell) = self.lstm(embedded)        return outputs, hidden, cell# SOLUTION: Attention Layerclass Attention(nn.Module):    def __init__(self, hidden_dim):        super(Attention, self).__init__()        self.attn = nn.Linear(hidden_dim * 2, hidden_dim)        self.v = nn.Linear(hidden_dim, 1, bias=False)        def forward(self, hidden, encoder_outputs):        # hidden: (1, batch_size, hidden_dim)        # encoder_outputs: (batch_size, src_len, hidden_dim)        batch_size = encoder_outputs.shape[0]        src_len = encoder_outputs.shape[1]                # Répéter hidden pour chaque timestep        hidden = hidden.repeat(src_len, 1, 1)  # (src_len, batch_size, hidden_dim)        hidden = hidden.permute(1, 0, 2)  # (batch_size, src_len, hidden_dim)                # Calcul des scores d'attention        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))        attention = self.v(energy).squeeze(2)  # (batch_size, src_len)                return torch.softmax(attention, dim=1)# SOLUTION: Decoder LSTM avec Attentionclass DecoderWithAttention(nn.Module):    def __init__(self, output_size, embedding_dim, hidden_dim):        super(DecoderWithAttention, self).__init__()        self.embedding = nn.Embedding(output_size, embedding_dim)        self.attention = Attention(hidden_dim)        self.lstm = nn.LSTM(embedding_dim + hidden_dim, hidden_dim, batch_first=True)        self.fc = nn.Linear(hidden_dim, output_size)        def forward(self, x, hidden, cell, encoder_outputs):        # x: (batch_size, 1)        embedded = self.embedding(x)  # (batch_size, 1, embedding_dim)                # Attention        attn_weights = self.attention(hidden, encoder_outputs)  # (batch_size, src_len)        attn_weights = attn_weights.unsqueeze(1)  # (batch_size, 1, src_len)                # Context vector        context = torch.bmm(attn_weights, encoder_outputs)  # (batch_size, 1, hidden_dim)                # Combiner embedding et context        lstm_input = torch.cat((embedded, context), dim=2)  # (batch_size, 1, emb+hidden)                # LSTM        output, (hidden, cell) = self.lstm(lstm_input, (hidden, cell))                # Prédiction        prediction = self.fc(output.squeeze(1))  # (batch_size, output_size)                return prediction, hidden, cell, attn_weightsprint("Seq2Seq architecture defined!")

In [ ]:
# SOLUTION: Implémenter l'entraînement et tester sur de nouveaux exemples# ASTUCE: Utiliser teacher forcing pendant l'entraînement# ASTUCE: Pour l'inférence, générer mot par mot jusqu'à <EOS>print("\n=== EXERCISE 1: Implement training and inference ===")print("Hint: Use teacher forcing ratio of 0.5")print("Hint: Train for ~50 epochs with batch_size=32")print("Expected accuracy: >95% on test set")

## Exercice 2: Génération de Texte avec LSTM### ObjectifCréer un générateur de texte caractère par caractère.### Instructions1. Utiliser un texte simple (ex: Shakespeare, Lorem Ipsum)2. Créer un vocabulaire de caractères3. Générer des séquences (input: N caractères, target: N+1ème caractère)4. Entraîner un LSTM5. Générer du nouveau texte avec sampling

In [ ]:
# SOLUTION: Texte d'exempletext = """To be or not to be, that is the question.Whether tis nobler in the mind to sufferThe slings and arrows of outrageous fortune,Or to take arms against a sea of troublesAnd by opposing end them.""" * 20  # Répéter pour avoir plus de données# SOLUTION: Créer vocabulaire de caractèreschars = sorted(list(set(text)))char_to_idx = {ch: i for i, ch in enumerate(chars)}idx_to_char = {i: ch for i, ch in enumerate(chars)}vocab_size = len(chars)print(f"Text length: {len(text)}")print(f"Vocab size: {vocab_size}")print(f"Characters: {''.join(chars)}")

In [ ]:
# SOLUTION: Créer séquencesseq_length = 40sequences = []next_chars = []for i in range(len(text) - seq_length):    sequences.append(text[i:i+seq_length])    next_chars.append(text[i+seq_length])print(f"Number of sequences: {len(sequences)}")print(f"Example: '{sequences[0]}' -> '{next_chars[0]}'")

In [ ]:
# SOLUTION: Modèle LSTM pour génération de caractèresclass CharLSTM(nn.Module):    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers):        super(CharLSTM, self).__init__()        self.embedding = nn.Embedding(vocab_size, embedding_dim)        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)        self.fc = nn.Linear(hidden_dim, vocab_size)        def forward(self, x, hidden=None):        embedded = self.embedding(x)        if hidden is None:            output, hidden = self.lstm(embedded)        else:            output, hidden = self.lstm(embedded, hidden)        output = self.fc(output)        return output, hidden# SOLUTION: Fonction de générationdef generate_text(model, start_str, length, temperature=1.0):    """Génère du texte à partir d'un seed.        temperature: contrôle la créativité (0.5=conservateur, 1.5=créatif)    """    model.eval()    chars_generated = [ch for ch in start_str]    input_seq = torch.LongTensor([[char_to_idx[ch] for ch in start_str]]).to(device)    hidden = None        with torch.no_grad():        for _ in range(length):            output, hidden = model(input_seq, hidden)            output = output[0, -1, :] / temperature            probs = torch.softmax(output, dim=0).cpu().numpy()            char_idx = np.random.choice(len(probs), p=probs)            next_char = idx_to_char[char_idx]            chars_generated.append(next_char)            input_seq = torch.LongTensor([[char_idx]]).to(device)        return ''.join(chars_generated)print("\n=== EXERCISE 2: Implement training and generate text ===")print("Hint: Train for ~100 epochs")print("Hint: Try different temperatures (0.5, 1.0, 1.5)")print("Expected: Generated text should resemble Shakespeare style")

## Exercice 3: Prédiction de Séries Temporelles avec GRU### ObjectifPrédire les valeurs futures d'une série temporelle (ex: sinusoïde + bruit).### Instructions1. Générer une série temporelle synthétique2. Créer des séquences (fenêtre glissante)3. Entraîner un GRU4. Prédire les valeurs futures5. Visualiser les prédictions

In [ ]:
# SOLUTION: Générer série temporelledef generate_time_series(n_points=1000):    t = np.linspace(0, 100, n_points)    # Combinaison de sinusoïdes + tendance + bruit    series = (        10 * np.sin(0.1 * t) +         5 * np.sin(0.3 * t) +         0.05 * t +        np.random.randn(n_points) * 2    )    return seriestime_series = generate_time_series(1000)plt.figure(figsize=(14, 4))plt.plot(time_series[:200], linewidth=2)plt.xlabel('Time', fontsize=12)plt.ylabel('Value', fontsize=12)plt.title('Time Series Data (first 200 points)', fontsize=14, fontweight='bold')plt.grid(True, alpha=0.3)plt.show()

In [ ]:
# SOLUTION: Créer séquences avec fenêtre glissantedef create_sequences(data, seq_length, pred_length=1):    """Créer des paires (X, y) pour prédiction multi-step."""    X, y = [], []    for i in range(len(data) - seq_length - pred_length + 1):        X.append(data[i:i+seq_length])        y.append(data[i+seq_length:i+seq_length+pred_length])    return np.array(X), np.array(y)seq_length = 50pred_length = 10  # Prédire 10 pas dans le futur# SOLUTION: Normaliser les données# SOLUTION: Créer train/test split# SOLUTION: Créer DataLoader PyTorchprint("\n=== EXERCISE 3: Implement GRU for time series prediction ===")print(f"Hint: Use sequence length = {seq_length}")print(f"Hint: Predict {pred_length} steps ahead")print("Expected: MSE < 5.0 on test set")

In [ ]:
# SOLUTION: Modèle GRUclass TimeSeriesGRU(nn.Module):    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):        super(TimeSeriesGRU, self).__init__()        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)        self.fc = nn.Linear(hidden_dim, output_dim)        def forward(self, x):        # x: (batch_size, seq_len, input_dim)        output, hidden = self.gru(x)        # Prendre le dernier output        prediction = self.fc(output[:, -1, :])        return prediction# SOLUTION: Entraîner et visualiser les prédictions

## Exercice 4: Classification Multi-classe avec BERT### ObjectifUtiliser BERT pour classifier des textes en plusieurs catégories.### Instructions1. Créer un dataset synthétique de 3 catégories (tech, sport, politique)2. Fine-tuner BERT pour classification multi-classe3. Évaluer avec accuracy et confusion matrix4. Tester sur de nouveaux textes

In [ ]:
# SOLUTION: Dataset multi-classetech_texts = [    "new smartphone release powerful processor",    "artificial intelligence machine learning breakthrough",    "software update bug fixes performance improvements",    "cloud computing data center expansion",    "cybersecurity threat detection prevention"] * 40sport_texts = [    "football match championship final victory",    "basketball team playoff tournament win",    "tennis player grand slam title",    "olympic games gold medal record",    "soccer world cup qualifier goal"] * 40politics_texts = [    "election campaign presidential debate vote",    "government policy reform legislation passed",    "international summit diplomatic relations treaty",    "parliament session bill proposal discussion",    "political party coalition agreement negotiation"] * 40# SOLUTION: Combiner et créer labels (0=tech, 1=sport, 2=politics)# SOLUTION: Utiliser transformers library pour BERT# SOLUTION: Fine-tuner et évaluerprint("\n=== EXERCISE 4: Implement multi-class classification with BERT ===")print("Hint: Use 'bert-base-uncased' model")print("Hint: Set num_labels=3")print("Expected accuracy: >90%")

## Solutions (à décommenter après avoir essayé)Les solutions sont fournies ci-dessous. Essayez d'abord de résoudre les exercices par vous-même!

In [ ]:
# SOLUTION EXERCICE 1: Disponible dans les notebooks de démonstration# SOLUTION EXERCICE 2: Voir 08_demo_lstm_sentiment.ipynb pour architecture similaire# SOLUTION EXERCICE 3: Adapter le modèle LSTM avec sortie multi-step# SOLUTION EXERCICE 4: Voir 08_demo_transformers_huggingface.ipynb

## Bonus: Projets Avancés### Projet 1: Chatbot Simple- Créer un dataset de paires question-réponse- Entraîner un seq2seq avec attention- Ajouter beam search pour génération### Projet 2: Résumé Automatique- Utiliser un dataset de textes longs + résumés- Fine-tuner BART ou T5- Évaluer avec ROUGE score### Projet 3: Named Entity Recognition- Dataset avec entités annotées (PER, LOC, ORG)- Fine-tuner BERT pour NER- Visualiser les entités détectées### Projet 4: Analyse de Sentiments Multi-aspects- Analyser différents aspects d'un produit (qualité, prix, service)- Multi-task learning avec BERT- Visualiser les sentiments par aspect

## Ressources Complémentaires### Papers Importants- Seq2Seq: "Sequence to Sequence Learning with Neural Networks" (Sutskever et al., 2014)- Attention: "Neural Machine Translation by Jointly Learning to Align and Translate" (Bahdanau et al., 2015)- Transformer: "Attention is All You Need" (Vaswani et al., 2017)- BERT: "BERT: Pre-training of Deep Bidirectional Transformers" (Devlin et al., 2018)### Datasets Publics- IMDB: Sentiment analysis (50k reviews)- WMT: Machine translation- SQuAD: Question answering- GLUE/SuperGLUE: NLP benchmarks### Bibliothèques- Hugging Face Transformers: https://huggingface.co/transformers/- PyTorch: https://pytorch.org/- spaCy: https://spacy.io/- NLTK: https://www.nltk.org/